# Ragas 对北京旅游 RAG 的评估

通过 LlamaIndex 模拟最小的 RAG，加载北海信息。

测试评估方式。

## 加载数据

In [2]:
%%time
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()
documents[0].text[:500]

CPU times: user 1.45 ms, sys: 189 µs, total: 1.64 ms
Wall time: 1.41 ms


'收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。 [3] [28]\n北海及团城于1961年被中华人民共和国中央人民政府公布为第一批全国重点文物保护单位。 [32]北海公园也是国家AAAA级旅游景区。 [33]\n2023年4月，北海公园入选第二批国家级文明旅游示范单位公示名单。 [37]\n- 中文名\n- 北海公园\n- 外文名\n- Beihai Park\n- 地理位置\n- 北京市西城区文津街1号\n- 气候条件\n- 温带季风气候\n- 开放时间\n- 4月1日-10月31日 06:00-21:00；11月1日-次年3月31日 06:30-20:00 [1]\n- 景点级别\n- AAAA级\n- 门票价格\n- 旺季10元，淡季5元\n- 占地面积\n- 682000 m²（其中水面积38.9万平方米，陆地面积29.3万平方米）\n- 著名景点\n- 琼华岛、永安寺、北海白塔\n北海公园原为辽燕京城东北一片湖泊，名金海。中有'

## 生成测试集

In [10]:
%%time

from datasets import Dataset 

data_samples = {
    'question': ['北海公园在哪里？', ],
    'contexts' : ['北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。'],
    'ground_truth': ['北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。'],
    
}

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 7.87 µs


In [11]:
import pandas as pd
from datasets import Dataset


# 创建Pandas DataFrame
df = pd.DataFrame(data_samples)

df.head()

,question,contexts,ground_truth
0,北海公园在哪里？,北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北...,北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。


In [12]:
hf_dataset = Dataset.from_pandas(df)
hf_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth'],
    num_rows: 1
})

## 创建 RAG

In [14]:
%%time

# build query engine
from llama_index.core import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model=OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

vector_index = VectorStoreIndex.from_documents(documents)

query_engine = vector_index.as_query_engine()

CPU times: user 1.16 s, sys: 36.5 ms, total: 1.2 s
Wall time: 9.82 s


In [15]:
df["question"][0]

'北海公园在哪里？'

In [16]:
%%time

response = query_engine.query(df["question"][0])
print(response)

北海公园位于北京市西城区文津街1号。
CPU times: user 57.8 ms, sys: 8.22 ms, total: 66 ms
Wall time: 2.27 s


## 评估

In [18]:
%%time

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 10.7 µs


In [19]:
%%time

# using GPT 3.5, use GPT 4 / 4-turbo for better accuracy
evaluator_llm = OpenAILike(
    model="gpt-3.5-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

CPU times: user 111 µs, sys: 0 ns, total: 111 µs
Wall time: 113 µs


In [24]:
data_samples["question"]

['北海公园在哪里？']

In [25]:
data_samples["ground_truth"]

['北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。']

In [26]:
%%time

from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=data_samples,
    llm=evaluator_llm,
    embeddings=Settings.embed_model,
)

Running Query Engine:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs


CPU times: user 250 ms, sys: 9.87 ms, total: 260 ms
Wall time: 8.52 s


In [27]:
result

{'faithfulness': 1.0000, 'answer_relevancy': 0.8781, 'context_precision': 1.0000, 'context_recall': 1.0000, 'harmfulness': 0.0000}

In [28]:
result.to_pandas()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,北海公园在哪里？,[[33]\n2023年4月，北海公园入选第二批国家级文明旅游示范单位公示名单。 [37]\...,北海公园位于北京市西城区文津街1号。,北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。,1.0,0.878119,1.0,1.0,0


In [35]:
result.scores.to_pandas()

,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,1.0,0.878119,1.0,1.0,0
